## Simple ML approaches to the Ames Housing regression problem

In this notebook I will explore simple regression techniques for the Ames Housing regression competition.
This models are simple but, through rigorous application, allow to obtain good results.

The notebook focuses on building complete pipelines for ML models:
* Preprocessing of data while avoiding information leakage:
    * We learn how to combine common feature preprocessing steps such as [1-hot encoding](https://en.wikipedia.org/wiki/One-hot#Machine_learning_and_statistics), [standardisation](https://en.wikipedia.org/wiki/Feature_scaling), dimensionality reduction via [principal component analysis](https://en.wikipedia.org/wiki/Principal_component_analysis) and [data imputation](https://en.wikipedia.org/wiki/Imputation_(statistics)).
    * We use sklearn's [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) and [`ColumnTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) to apply different techniques to subsets of the input columns.
    * We transform the label using a [quantile transformation](https://stats.stackexchange.com/questions/325570/quantile-transformation-with-gaussian-distribution-sklearn-implementation).
* Applying (simple) Machine Learning models:
    * A [linear regression model](https://en.wikipedia.org/wiki/Linear_regression)
    * A linear regression model, with [Lasso](https://en.wikipedia.org/wiki/Lasso_(statistics)) regularisation.
    * An [AdaBoost](https://en.wikipedia.org/wiki/AdaBoost) regressor using random trees.
    * An [XGBoost](https://en.wikipedia.org/wiki/XGBoost) regressor.
    * A [CatBoost](https://en.wikipedia.org/wiki/Catboost) regressor which, different from most other tree-based regressors, can use categorical features natively.
    * A [stacked model](https://en.wikipedia.org/wiki/Ensemble_learning#Stacking) using all the previous ones.

In [ ]:
# Ignore some FutureWarning, mainly from Pandas and Sklearn
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np

from sklearn.base import RegressorMixin, TransformerMixin
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, RobustScaler, StandardScaler, QuantileTransformer, FunctionTransformer
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_log_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, StackingRegressor

from xgboost import XGBRegressor
from catboost import CatBoostRegressor

### Reading and basic cleaning of the data

In [ ]:
categorical_features = [
    'MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape',
    'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
    'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
    'HouseStyle', 'OverallQual', 'OverallCond', 'RoofStyle',
    'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
    'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
    'BsmtExposure', 'BsmtFinType1',  'BsmtFinType2', 'Heating',
    'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 
    'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish',
    'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence',
    'MiscFeature', 'SaleType', 'SaleCondition'
]
numerical_features = [
    'LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
    'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF',
    '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath',
    'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
    'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
    'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
    'MiscVal', 'MoSold', 'YrSold'
]
label = 'SalePrice'

In [ ]:
def read_data(filename: str, remove_id: bool = True) -> pd.DataFrame:
    """ Reads a csv file with data and performs basic data cleaning.
        It only cleans those columns which can be cleaned without leading
        to information leakage. For example, it does not perform imputation.
        
        Parameters:
            filename: path of the csv file
            remove_id: if True, it removes the Id column
        
        Returns:
            A pandas dataframe with the processed dataset.
    """
    
    d = pd.read_csv(filename)
    
    # Replace numerical values with meaningful categorical ones for
    # column MSSubClass
    d.MSSubClass.replace(to_replace={
        20: '1StoryNew',
        30: '1StoryOld',
        40: '1StoryAttic',
        45: '1HalfStoryUnfinished',
        50: '1HalfStoryFinished',
        60: '2StoriesNew',
        70: '2StoriesOld',
        75: '2HalfStories',
        80: 'MultiLevel',
        85: 'SplitFoyer',
        90: 'Duplex',
        120: '1StoryPud',
        150: '1HalfStoryPud',
        160: '2StoriesPud',
        180: 'MultiLevelPud',
        190: '2Family'
    }, inplace=True)
    
    # Here I assume that a value of 'NA' for LotFrontage means that the
    # house is not directly connected to the street. In this case I can
    # replace the value with 0. Another option is that the data is actually
    # missing but, given that the rest of the dataset is quite clean,
    # I lean towards the first hypothesis.
    d.LotFrontage.fillna(value=0, inplace=True)
    
    # MasVnrArea has NaN instead of 0 in some rows corresponding to houses
    # without masonry veener.
    d.MasVnrArea.fillna(value=0, inplace=True)
    
    # Remove column Id which is just noise.
    if remove_id and 'Id' in d.columns:
        d.drop(columns='Id', inplace=True)
    
    # Mark categorical features with the appropriate Pandas type.
    for feature in categorical_features:
        d[feature] = pd.Categorical(d[feature])
    
    return d

In [ ]:
d = read_data(filename='data.csv')

### Model selection procedure

I use a [holdout validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics)#Holdout_method) method to select among different models and [$k$-fold cross-validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics)#k-fold_cross-validation) to perform hyperparameter tuning.

Therefore, I first split the data into two parts (houldout method).
I will perform hyperparameter tuning on the training (and validation) part, i.e., `X_train` and `y_train`.
I will then compare model performance on the test part, i.e., `X_test` and `y_test`.

In [ ]:
features = [column for column in d.columns if column != label]
X, y = d[features], d[[label]]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

### Creating pipelines for our models

Function `pipeline_for` below will be the main function I use to create pipelines for our models.
It takes an estimator and a number of parameters which define the preprocessing of the data, and combines preprocessing+estimator into a `Pipeline` object.
It uses the two functions `categorical_transformations` and `numerical_transformations` which, in turn, build the transformation steps for categorical and numerical features, respectively.

In [ ]:
def categorical_transformations(
    one_hot_encode: bool = True,
    one_hot_encode_scale: bool = True,
    scaler: TransformerMixin = RobustScaler()
) -> Pipeline:
    """ Creates a transformation pipeline for categorical features, to be passed
        to a ColumnTransformer. It always performs at least data imputation.
        
        Parameters:
            one_hot_encode: whether to 1-hot encode the features
            one_hot_encode_scale: whether to scale the 1-hot encoded features
            scaler: the scaler to use, in case scaling should be applied
            
        Returns:
            A pipeline with the preprocessing steps.
    """

    categorical_steps = [
        ('imputation', SimpleImputer(strategy='constant', fill_value='None'))
    ]
    
    if one_hot_encode:
        # I use handle_unknown='ignore' to make sure no error is raised if, at
        # prediction time, the encoder finds a value which it had never seen at
        # training time. In this case, using 'ignore' forces the encoder to put
        # all 0's in the corresponding 1-hot encoded columns.
        categorical_steps.append(
            ('one_hot_encoding', OneHotEncoder(handle_unknown='ignore', sparse=False)))

        if one_hot_encode_scale:
            # Scale the 1-hot encoded features
            categorical_steps.append(('one_hot_scaling', scaler))
        
    return Pipeline(steps=categorical_steps)

def numerical_transformations(
    numerical_scale: bool = True,
    scaler: TransformerMixin = RobustScaler()
) -> Pipeline:
    """ Creates a transformation pipeline for numerical features, to be passed
        to a ColumnTransformer. It always performs at least data imputation.
        
        Parameters:
            numerical_scale: whether to scale the features
            scaler: the scaler to use, in case scaling should be applied
            
        Returns:
            A pipeline with the preprocessing steps.
    """
    
    numerical_steps = [('imputation', SimpleImputer(strategy='median'))]

    if numerical_scale:
        numerical_steps.append(('scaling', scaler))
    
    return Pipeline(steps=numerical_steps)

In [ ]:
def pipeline_for(
    model: RegressorMixin,
    one_hot_encode: bool = True,
    one_hot_encode_scale: bool = True,
    numerical_scale: bool = True,
    scaler: TransformerMixin = RobustScaler(),
    pca: bool = False
) -> Pipeline:
    """ Creates a pipeline to use the data with a variety of models.
    
        Parameters:
            d: the pandas dataframe to use
            model: prediction model to add at the end of the pipeline
            one_hot_encode: whether to 1-hot encode categorical features
            one_hot_encode_scaler: whether 0/1 1-hot encoded features will also be scaled
            numerical_scale: whether to scale numerical features
            scaler: scaler object to use for scaling columns
            pca: whether to reduce the number of features via Principal Component Analysis before calling the model
            
        Returns:
            An sklearn pipeline. It can be trained directly or, if using PCA, passed to a cross-validation
            method to tune the number of components. The corresponding parameter to use in the param grid
            is 'pca__n_components'. Model hyperparameters to use in the param grid will be 'model__<...>'.
    """
    
    transformers = [
        ('categorical',
         categorical_transformations(one_hot_encode, one_hot_encode_scale, scaler),
         categorical_features),
        ('numerical',
         numerical_transformations(numerical_scale, scaler),
         numerical_features)
    ]
        
    preprocessor = ColumnTransformer(transformers=transformers)
    
    # After preprocessing the input feature, one might want to try to
    # reduce their number using Principal Component Analysis. In this
    # case, one should first append a PCA object to the Preprocessor.
    # Otherwise, directly append the model.
    
    steps = [('preprocessing', preprocessor)]
    
    if pca:
        steps.append(('pca', PCA()))
        
    steps.append(('model', model))
    
    return Pipeline(steps=steps)

## Example 1: Multiple Linear Regression

In this example I use:
* A transformation pipeline with imputing and 1-hot encoding of categorical column, imputing of numerical columns, and scaling of all columns (using a [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)).
* Principal component analysis using [`PCA`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html). We choose the best number of components via cross-validation (hence the use of `GridSearchCV`), trying values $\{15, 15, 20, 25, \ldots, 80\}$.
* A linear regression model, using [`LinearRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html).
* Target transformation, using [`QuantileTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.QuantileTransformer.html) with a target normal distribution and using 900 quantiles.

In [ ]:
linear_model = pipeline_for(
    model=TransformedTargetRegressor(
        regressor=LinearRegression(),
        transformer=QuantileTransformer(n_quantiles=900, output_distribution='normal')
    ),
    scaler=StandardScaler(),
    pca=True
)

I now use 5-fold cross-validation to select the best PCA hyperparameter (the number of principal components).
I validate using the mean squared log error, because this is the intrelevant if we want to participate in the [description of the Kaggle competition](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview/evaluation).

In [ ]:
linear_model_gs = GridSearchCV(
    estimator=linear_model,
    param_grid={
        'pca__n_components': range(15, 85, 5)
    },
    scoring='neg_mean_squared_log_error',
    n_jobs=4,
    cv=5
)
linear_model_gs.fit(X_train, y_train)

In [ ]:
linear_model_gs.best_params_

#### Estimating the quality of the model using the test set

In [ ]:
predictions = linear_model_gs.predict(X_test)

In [ ]:
mean_squared_log_error(y_true=y_test, y_pred=predictions)

## Example 2: Multiple Linear Regression with LASSO Regularisation

This model will look similar to the previous one, except that:
1. I apply a Lasso regularsation term and, therefore, I must tune its hyperparameter $\alpha$.
2. Because Lasso performs implicit feature selection, I do not use PCA.

The grid search to find the best value of $\alpha$ uses a log-space with 20 entries between $10^{-6}$ and $10^1$.
I use 5-fold CV to tune this hyperparameter.

In [ ]:
# I use max_iter=10000 because Lasso's default 1000 are usually
# not enought to ensure convergence.
lasso = pipeline_for(
    model=TransformedTargetRegressor(
        regressor=Lasso(max_iter=10000),
        transformer=QuantileTransformer(n_quantiles=900, output_distribution='normal')
    )
)

In [ ]:
lasso_gs = GridSearchCV(
    estimator=lasso,
    param_grid={
        'model__regressor__alpha': np.logspace(start=-6, stop=1, num=20)
    },
    scoring='neg_mean_squared_log_error',
    n_jobs=4,
    cv=5
)
lasso_gs.fit(X_train, y_train)

In [ ]:
lasso_gs.best_params_

#### Estimating the quality of the model using the test set

In [ ]:
predictions = lasso_gs.predict(X_test)

In [ ]:
mean_squared_log_error(y_true=y_test, y_pred=predictions)

## Example 3: Ada Boost

Here I will be using an Ada(ptive)Boost method, with a simple decision tree regressor as the base estimator.
Sklearn's [`DecisionTreeRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html) has quite a number of hyperparameter.
Here I only tune one (`max_depth`); moreover, I tune two hyperparameters of `AdaBoostRegressor`, namely `n_estimators` and `learning_rate`.

I will 1-hot encode the categorical features, because `DecisionTreeRegressor` is not able to deal with categorical features natively.
I will not scale the features, as I don't see much point in doing so with a tree-based estimator (it amounts to just a rescaling of the splitting thresholds).

In [ ]:
ada_boost = pipeline_for(
    model=AdaBoostRegressor(base_estimator=DecisionTreeRegressor()),
    one_hot_encode_scale=False,
    numerical_scale=False
)

In [ ]:
ada_boost_gs = GridSearchCV(
    estimator=ada_boost,
    param_grid={
        'model__n_estimators': [10, 50, 100, 500, 1000],
        'model__learning_rate': [0.001, 0.01, 0.1, 1],
        'model__base_estimator__max_depth': range(5, 11)
    },
    scoring='neg_mean_squared_log_error',
    n_jobs=4,
    cv=5
)
ada_boost_gs.fit(X_train, y_train.values.ravel())

In [ ]:
ada_boost_gs.best_params_

#### Estimating the quality of the model using the test set

In [ ]:
predictions = ada_boost_gs.predict(X_test)

In [ ]:
mean_squared_log_error(y_true=y_test, y_pred=predictions)

## Example 4: XGBoost

Next, trying an [`XGBRegressor`](https://xgboost.readthedocs.io/en/stable/python/python_api.html).
Note how this model also doesn't (yet) support categorical features and so I must 1-hot encode them.

In [ ]:
xgb = pipeline_for(
    model=XGBRegressor(eval_metric=mean_squared_log_error),
    one_hot_encode=True,
    one_hot_encode_scale=False,
    numerical_scale=False
)

In [ ]:
xgb_cv = GridSearchCV(
    estimator=xgb,
    param_grid={
        'model__n_estimators': [10, 50, 100, 500, 1000],
        'model__learning_rate': [0.001, 0.01, 0.1, 1],
        'model__max_depth': range(5, 11)
    },
    scoring='neg_mean_squared_log_error',
    n_jobs=1,
    cv=5,
    verbose=4
)
xgb_cv.fit(X_train, y_train)

In [ ]:
xgb_cv.best_params_

#### Estimating the quality of the model using the test set

In [ ]:
predictions = xgb_cv.predict(X_test)

In [ ]:
mean_squared_log_error(y_true=y_test, y_pred=np.maximum(predictions, 0))

## Example 5: CatBoost

Finally, we use [`CatBoostRegressor`](https://catboost.ai/en/docs/concepts/python-reference_catboostregressor).
Because this model supports categorical features, we turn off 1-hot encoding.

In [ ]:
catb = pipeline_for(
    model=CatBoostRegressor(cat_features=list(range(len(categorical_features)))),
    one_hot_encode=False,
    one_hot_encode_scale=False,
    numerical_scale=False
)

In [ ]:
catb.fit(X_train, y_train)

In [ ]:
predictions = catb.predict(X_test)

In [ ]:
mean_squared_log_error(y_true=y_test, y_pred=predictions)

## Building a stacked regressor with all the models above

I now want to build a stacked regressor, combining all the models I tried until now and using `CatBoostRegressor` as the meta-estimator.
This is no trivial task, because each regressor requires different preprocessing steps which, ultimately, feed the estimator with different sets of features.
It will use the best hyperparameters tuned above, so I will not perform any hyperparameter tuning on the stacked regressor.

The main pipeline to use with [`StackingRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingRegressor.html) will be the following:
1. Perform data imputation, as this is a step required by all models.
2. `SimpleImputer` transforms the pandas dataframe it receive in input into a numpy array.
This is not convenient, because later preprocessing steps used by the models rely on the data to be in `DataFrame` format (for example, because they refer to the columns by name).
Therefore, I add a `back_to_pandas` transformation which brings the numpy values back into `DataFrame` form.
3. Finally, the model itself `StackingRegressor` with all its estimators and the meta-estimator.
Each of the estimators will take care of those preprocessing steps which are peculiar to its model.

In [ ]:
def back_to_pandas(values, imputer) -> pd.DataFrame:
    """ Transform raw X values back into a pandas DataFrame.
    
        Parameters:
            values: the X values as a numpy 2d array
            imputer: the imputer object which generated the X values and retains the column names
            
        Returns:
            A pandas DataFrame with the data (including the imputed values) and proper column names
    """
    
    df = pd.DataFrame(values, columns=imputer.get_feature_names_out())
    for column in df.columns:
        if 'numerical' in column:
            df[column] = df[column].astype(np.float64)
        elif 'categorical' in column:
            df[column] = pd.Categorical(df[column])
    return df

class CatBoostRegressorAutoDetectCategorical(CatBoostRegressor):
    """ A modified version of CatBoostRegressor which automatically detects categorical features.
        If the fit() method is called with a pandas DataFrame, it detects using the column dtype.
        Otherwise, it attempts to convert values to float. It marks columns in which it fails as
        categorical columns.
    """
    
    def fit(self, X, y=None, **fit_params):
        if hasattr(X, 'columns'):
            cat = [column for column in X.columns if X[column].dtype == 'category']
            return super().fit(X, y, cat_features=cat, **fit_params)
        
        cat_cols = set()
        for row in X:
            for col, val in enumerate(row):
                try:
                    float(val)
                except ValueError:
                    cat_cols.add(col)
        
        return super().fit(X, y, cat_features=list(sorted(cat_cols)), **fit_params)

### Preprocessing: Imputation
# Impute a constant 'None' for missing categorical values.
# Impute the median for missing numerical values.
sr_imputer = ColumnTransformer(transformers=[
    ('sr_imputer_categorical',
        SimpleImputer(strategy='constant', fill_value='None'),
        make_column_selector(dtype_include='category')),
    ('sr_imputer_numerical',
        SimpleImputer(strategy='median'),
        make_column_selector(dtype_include=np.number))
], verbose=True)

### Preprocessing: 1-hot encoding
# Encode categorical columns.
# Skip numerical columns.
sr_encoder = ColumnTransformer(transformers=[
    ('sr_econder_categorical',
        OneHotEncoder(sparse=False, handle_unknown='ignore'),
        make_column_selector(dtype_include='category')),
    ('sr_encoder_numerical',
        'passthrough',
        make_column_selector(dtype_exclude='category'))
], verbose=True)

### Preprocessing back-to-pandas
# Get back a DataFrame after imputation
sr_back_to_pandas = FunctionTransformer(
    func=lambda values: back_to_pandas(values, sr_imputer)
)

### Linear model (with PCA and target transformation)
sr_linear_inner = TransformedTargetRegressor(
    regressor=make_pipeline(PCA(n_components=50), LinearRegression()),
    transformer=QuantileTransformer(n_quantiles=900, output_distribution='normal')
)

### Pipeline for the linear model (with preprocessing)
sr_linear = Pipeline(steps=[
    ('sr_linear_one_hot_encoding', sr_encoder),
    ('sr_linear_scaling', StandardScaler()),
    ('sr_linear_model', sr_linear_inner)
], verbose=True)

### Lasso model (with target transformation)
sr_lasso_inner = TransformedTargetRegressor(
    regressor=Lasso(alpha=0.0008858667904100823, max_iter=10000),
    transformer=QuantileTransformer(n_quantiles=900, output_distribution='normal')
)

### Pipeline for the lasso model (with preprocessing)
sr_lasso = Pipeline(steps=[
    ('sr_lasso_one_hot_encoding', sr_encoder),
    ('sr_lasso_scaling', RobustScaler()),
    ('sr_lasso_model', sr_lasso_inner)
], verbose=True)

### Adaboost model
sr_adaboost_inner = AdaBoostRegressor(
    base_estimator=DecisionTreeRegressor(max_depth=10),
    learning_rate=1,
    n_estimators=1000
)

### Pipeline for the Adaboost model (with preprocessing)
sr_adaboost = Pipeline(steps=[
    ('sr_adaboost_one_hot_encoding', sr_encoder),
    ('sr_adaboost_model', sr_adaboost_inner)
], verbose=True)


### XGBoost model
sr_xgboost_inner = XGBRegressor(
    n_estimators=1000,
    learning_rate=0.01,
    max_depth=5)

### Pipeline for the XGBoost model (with preprocessing)
sr_xgboost = Pipeline(steps=[
    ('sr_xgboost_one_hot_encoding', sr_encoder),
    ('sr_xgboost_model', sr_xgboost_inner)
], verbose=True)

### Catboost model (does not need any model-specifing preprocessing)
sr_catboost = CatBoostRegressorAutoDetectCategorical()

### Stacking regressor model
sr_stacking_regressor = StackingRegressor(
    estimators=[
        ('sr_stacking_regressor_linear_regression', sr_linear),
        ('sr_stacking_regressor_lasso_regression', sr_lasso),
        ('sr_stacking_regressor_ada_boost', sr_adaboost),
        ('sr_stacking_regressor_xgboost', sr_xgboost),
        ('sr_stacking_regressor_catboost', sr_catboost)
    ],
    final_estimator=CatBoostRegressorAutoDetectCategorical(),
    passthrough=True,
    verbose=3
)

### Pipeline for the stacking regressor model (with preprocessing, i.e., data imputation)
stacking_regressor = Pipeline(steps=[
    ('stacking_regressor_imputation', sr_imputer),
    ('stacking_regressor_back_to_pandas', sr_back_to_pandas),
    ('stacking_regressor_model', sr_stacking_regressor)
], verbose=True)

In [ ]:
stacking_regressor.fit(X_train, y_train.values.ravel())

In [ ]:
predictions = stacking_regressor.predict(X_test)

In [ ]:
mean_squared_log_error(y_true=y_test, y_pred=predictions)

## Retraining the winning model on the entire dataset

In [ ]:
stacking_regressor.fit(X, y)